In [3]:
!pip install efficientnet_pytorch
!pip install albumentations
!pip install segmentation_models_pytorch

  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60945 sha256=9b266ad661db133e0729fc7523c431547bec6e21f92af7bdeefb5b16fb2d4872
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built pretrainedmodels


In [5]:
!unzip -qq "/content/drive/MyDrive/open.zip"

In [1]:
import os
import cv2
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.optim as optim

from tqdm import tqdm
import albumentations as A
from albumentations import (
    Resize,
    Compose,
    HorizontalFlip,
    VerticalFlip,
    Normalize,
)
from efficientnet_pytorch import EfficientNet
import segmentation_models_pytorch as smp

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
    model = smp.Unet(
        encoder_name="efficientnet-b4",
        encoder_weights="imagenet",
        in_channels=3,
        classes=1,
    ).to(device)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [5]:
def get_training_augmentation():
    train_transform = [
        Resize(1024, 1024),
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.5),
        Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), p=1),
    ]

    return Compose(train_transform, p=1)

class SatelliteDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 1]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image

        mask_rle = self.data.iloc[idx, 2]
        mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = np.transpose(augmented['image'], (2, 0, 1))  # Change image from (H, W, C) to (C, H, W)
            mask = np.expand_dims(augmented['mask'], axis=0)  #  Change mask from (H, W) to (1, H, W)

        return image, mask

dataset = SatelliteDataset(csv_file='/content/train.csv', transform=get_training_augmentation())
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=4)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize model
model = smp.Unet(
    encoder_name="efficientnet-b4",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
).to(device)  # Move model to the device

# Define loss function
loss = nn.BCEWithLogitsLoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Start training loop
for epoch in range(10):  # 10 epochs
    model.train()
    epoch_loss = 0
    for i,(images, masks) in enumerate(dataloader):
        images = images.float().to(device)
        masks = masks.float().to(device)

        # Forward pass
        outputs = model(images)
        cost = loss(outputs, masks)

        # Backward pass and optimization
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch + 1}/10], Step [{i + 1}/{len(dataloader)}], Loss: {cost.item()}")

    # Save model every 10 epochs
    if (epoch + 1) % 10 == 0:
        torch.save(model.state_dict(), f"model_epoch_{epoch + 1}.pth")


Epoch [1/10], Step [100/3570], Loss: 0.08102641999721527
Epoch [1/10], Step [200/3570], Loss: 0.1146855428814888
Epoch [1/10], Step [300/3570], Loss: 0.04809413105249405
Epoch [1/10], Step [400/3570], Loss: 0.07174326479434967
Epoch [1/10], Step [500/3570], Loss: 0.06325893849134445
Epoch [1/10], Step [600/3570], Loss: 0.06368810683488846
Epoch [1/10], Step [700/3570], Loss: 0.028474869206547737
Epoch [1/10], Step [800/3570], Loss: 0.06439483165740967
Epoch [1/10], Step [900/3570], Loss: 0.028462529182434082
Epoch [1/10], Step [1000/3570], Loss: 0.11030863225460052
Epoch [1/10], Step [1100/3570], Loss: 0.027807846665382385
Epoch [1/10], Step [1200/3570], Loss: 0.038018763065338135
Epoch [1/10], Step [1300/3570], Loss: 0.05161105468869209
